<a href="https://colab.research.google.com/github/germanloera/05MAIR---Algoritmos-de-Optimizacion---2019/blob/master/German_Loera_AG3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Actividad Guiada 3 - AG3 <br>
Germán Loera <br>
Url: https://colab.research.google.com/drive/1-KF1OkPmebpGITTuPj2C_waVCvd2yFiB

In [0]:
!pip install tsplib95

In [0]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)


('swiss42.tsp', <http.client.HTTPMessage at 0x7f3d7c48c4e0>)

In [0]:
import tsplib95
import random
import math


problem = tsplib95.load_problem(file)
Nodos = list(problem.get_nodes())
Aristas = list ( problem.get_edges())


# Se genera una solucion aleatoria con comienzo en el nodo 0
def crear_solucion(Nodos):
  solucion = [0]
  for i in range(len(Nodos) - 1):
    solucion = solucion + [ random.choice(list(set(Nodos)  - set(solucion)))]
  return solucion

# Devuelve la distancia entre 2 nodos
def distancia(a, b , problem):
  return problem.wfunc(a,b)

# Devuelve la distancia total de una trayectoria/solucion

def distancia_total(solucion, problema):
  distancia_total = 0
  for i in range(len(solucion) - 1 ):
    distancia_total += distancia(solucion[i], solucion[i +1 ], problem)
  return distancia_total + distancia ( solucion[len(solucion) - 1], solucion[0], problem)


In [0]:
# Busqueda Aleatoria
def busqueda_aleatoria(problem, N):
  mejor_solucion = crear_solucion(Nodos)
  mejor_valor = distancia_total(mejor_solucion, problem)

  for i in range(N):
    solucion = crear_solucion(Nodos)
    valor = distancia_total(solucion, problem)
    if valor < mejor_valor:
      mejor_valor = valor
      mejor_solucion = solucion
  print("la mejor solucion es ", mejor_valor, mejor_solucion)

busqueda_aleatoria(problem, 50000)

la mejor solucion es  3247 [0, 28, 5, 15, 7, 17, 18, 29, 31, 38, 25, 13, 11, 26, 10, 41, 23, 9, 22, 21, 1, 16, 37, 35, 36, 3, 6, 12, 19, 32, 34, 20, 33, 30, 40, 8, 39, 24, 4, 14, 2, 27]


In [0]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion
      
solucion = crear_solucion(Nodos)
mejor_solucion = genera_vecina(solucion)

In [0]:
#####################################################################
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion(Nodos)
  
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
        
    solucion_referencia = vecina
  
  print("La mejor solución encontrada es " , mejor_distancia , mejor_solucion)

sol = busqueda_local(problem, 100)


La mejor solución encontrada es  1756 [0, 27, 2, 28, 29, 30, 38, 34, 33, 20, 31, 17, 37, 15, 16, 14, 7, 32, 1, 6, 4, 3, 8, 10, 18, 13, 19, 36, 35, 22, 39, 24, 40, 21, 9, 23, 41, 25, 11, 12, 26, 5]


In [0]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina

solucion = crear_solucion(Nodos)
print(solucion, "\n ", genera_vecina_aleatorio(solucion))

[0, 38, 20, 35, 33, 25, 11, 26, 5, 10, 29, 4, 30, 24, 2, 31, 1, 36, 17, 9, 40, 18, 15, 12, 27, 6, 41, 34, 3, 22, 37, 7, 8, 28, 19, 23, 32, 21, 13, 16, 39, 14] 
  [0, 38, 20, 35, 33, 25, 11, 26, 5, 10, 29, 4, 30, 24, 2, 31, 1, 36, 17, 9, 40, 18, 15, 12, 7, 6, 41, 34, 3, 22, 37, 27, 8, 28, 19, 23, 32, 21, 13, 16, 39, 14]


In [0]:
#    a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False


def bajar_temperatura(T):
  return T*.9999


In [0]:
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100

  while TEMPERATURA > 1:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion  )
  return mejor_solucion

sol = recocido_simulado(problem, TEMPERATURA=10000)



	****Encontrada mejor solucion: 4457 [0, 22, 24, 28, 38, 17, 31, 21, 8, 2, 12, 30, 34, 3, 23, 35, 15, 39, 9, 1, 16, 6, 7, 5, 40, 19, 14, 33, 36, 32, 29, 26, 27, 41, 25, 10, 13, 37, 11, 4, 20, 18] 

	****Encontrada mejor solucion: 4434 [0, 26, 6, 28, 38, 17, 31, 21, 8, 41, 12, 30, 34, 3, 23, 11, 15, 39, 9, 1, 16, 24, 7, 5, 40, 19, 14, 33, 36, 32, 29, 22, 27, 2, 25, 10, 13, 37, 35, 4, 20, 18] 

	****Encontrada mejor solucion: 4364 [0, 26, 6, 28, 38, 11, 20, 21, 8, 41, 12, 30, 15, 19, 23, 16, 34, 39, 9, 1, 40, 24, 7, 5, 17, 3, 14, 33, 36, 32, 29, 22, 27, 2, 25, 10, 13, 37, 35, 4, 31, 18] 

	****Encontrada mejor solucion: 4332 [0, 37, 22, 19, 35, 41, 11, 38, 34, 3, 12, 13, 30, 29, 28, 31, 5, 2, 10, 25, 33, 18, 23, 24, 8, 27, 1, 6, 4, 40, 9, 21, 32, 16, 39, 15, 20, 17, 36, 14, 7, 26] 

	****Encontrada mejor solucion: 4184 [0, 37, 22, 19, 26, 41, 11, 38, 34, 3, 12, 13, 30, 29, 28, 31, 5, 2, 10, 25, 33, 18, 23, 24, 8, 27, 1, 6, 4, 40, 9, 21, 32, 16, 39, 15, 20, 17, 36, 14, 7, 35] 

	****Encon

In [0]:
# H - Hormiga, T - Feromonas
def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )


def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T





# Algoritmo colonia de hormigas

def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
    #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]

  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)


hormigas(problem, 5000)



[0, 17, 7, 13, 18, 40, 21, 27, 37, 9, 36, 28, 33, 34, 16, 14, 15, 19, 26, 4, 11, 29, 41, 23, 24, 6, 25, 30, 22, 1, 31, 32, 35, 38, 12, 10, 5, 3, 2, 20, 8, 39]
3903
